In [74]:

import gc
import polars as pl
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import seaborn as sns
import pandas as pd
from collections import Counter
from heapq import nlargest



df_train = pl.read_parquet('../data/train.parquet')
df_test = pl.read_parquet('../data/test.parquet')

In [31]:
# Load in a sample to a pandas df

sample_size = 150

chunks = pd.read_json("../data/test.jsonl", lines=True, chunksize = sample_size)

for c in chunks:
    sample_test_df = c
    print(c)
    break

      session                                             events
0    12899779  [{'aid': 59625, 'ts': 1661724000278, 'type': '...
1    12899780  [{'aid': 1142000, 'ts': 1661724000378, 'type':...
2    12899781  [{'aid': 141736, 'ts': 1661724000559, 'type': ...
3    12899782  [{'aid': 1669402, 'ts': 1661724000568, 'type':...
4    12899783  [{'aid': 255297, 'ts': 1661724000572, 'type': ...
..        ...                                                ...
145  12899924  [{'aid': 195179, 'ts': 1661724032545, 'type': ...
146  12899925  [{'aid': 526157, 'ts': 1661724032999, 'type': ...
147  12899926  [{'aid': 76309, 'ts': 1661724033017, 'type': '...
148  12899927  [{'aid': 1127083, 'ts': 1661724034268, 'type':...
149  12899928  [{'aid': 805848, 'ts': 1661724034631, 'type': ...

[150 rows x 2 columns]


In [38]:
for e, c in enumerate(chunks):
    print("this is e: " + str(e))
    print("this is c: " + str(c))
    if e > 4:
        break
    

this is e: 0
this is c:         session                                             events
600000   600000  [{'aid': 1448212, 'ts': 1659359041479, 'type':...
600001   600001  [{'aid': 585386, 'ts': 1659359041589, 'type': ...
600002   600002  [{'aid': 311893, 'ts': 1659359041621, 'type': ...
600003   600003  [{'aid': 687933, 'ts': 1659359041665, 'type': ...
600004   600004  [{'aid': 195812, 'ts': 1659359041690, 'type': ...
...         ...                                                ...
749995   749995  [{'aid': 1437146, 'ts': 1659367111303, 'type':...
749996   749996  [{'aid': 1457846, 'ts': 1659367111330, 'type':...
749997   749997  [{'aid': 179604, 'ts': 1659367111348, 'type': ...
749998   749998  [{'aid': 613440, 'ts': 1659367111406, 'type': ...
749999   749999  [{'aid': 118034, 'ts': 1659367111438, 'type': ...

[150000 rows x 2 columns]
this is e: 1
this is c:         session                                             events
750000   750000  [{'aid': 975580, 'ts': 1659367111446,

In [33]:
print(chunks)

In [34]:
sample_test_df.head()

,session,events
0,12899779,"[{'aid': 59625, 'ts': 1661724000278, 'type': '..."
1,12899780,"[{'aid': 1142000, 'ts': 1661724000378, 'type':..."
2,12899781,"[{'aid': 141736, 'ts': 1661724000559, 'type': ..."
3,12899782,"[{'aid': 1669402, 'ts': 1661724000568, 'type':..."
4,12899783,"[{'aid': 255297, 'ts': 1661724000572, 'type': ..."


In [35]:
sample_submission = pd.read_csv("../data/sample_submission.csv")
sample_submission.head()

,session_type,labels
0,12899779_clicks,129004 126836 118524
1,12899779_carts,129004 126836 118524
2,12899779_orders,129004 126836 118524
3,12899780_clicks,129004 126836 118524
4,12899780_carts,129004 126836 118524


Lets find the most common article for each different type of action

In [36]:
sample_size = 150000

chunks = pd.read_json("../data/train.jsonl", lines=True, chunksize = sample_size)

clicks_article_list = []
carts_article_list = []
orders_article_list = []

for e, c in enumerate(chunks):
    
    # Save time by not using all the data
    if e > 2:
        break
    
    sample_train_df = c
    
    for i, row in c.iterrows():
        actions = row['events']
        for action in actions:
            if action['type'] == 'clicks':
                clicks_article_list.append(action['aid'])
            elif action['type'] == 'carts':
                carts_article_list.append(action['aid'])
            else:
                orders_article_list.append(action['aid'])

In [37]:
#chunks = df_train.sample(n=150000, seed=42)

In [42]:
# Create dictionaries with articles and their frequencies
article_click_freq = Counter(clicks_article_list)
article_carts_freq = Counter(carts_article_list)
article_order_freq = Counter(orders_article_list)

In [92]:
top_click_article = sorted(article_click_freq, key=article_click_freq.get, reverse=True)[:20]
top_carts_article = sorted(article_carts_freq, key=article_carts_freq.get, reverse=True)[:20]
top_order_article = sorted(article_order_freq, key=article_order_freq.get, reverse=True)[:20]

In [91]:
#top_click_article

In [93]:
# Create a dict with this info
frequent_articles = {'clicks': top_click_article, 'carts':top_carts_article, 'order':top_order_article}

In [94]:
for action in ['clicks', 'carts', 'order']:
    print(f'Most frequent articles for {action}: {frequent_articles[action][:5]}')

Most frequent articles for clicks: [29735, 1733943, 108125, 1603001, 832192]
Most frequent articles for carts: [80222, 29735, 1733943, 166037, 485256]
Most frequent articles for order: [80222, 166037, 1022566, 351335, 1629608]


y_test = week nr 5 
y_train = weeks 1-4

y= each step in a session after truncation by timestamp (not done yet by the costumer in real session)
x= every step in the session, before truncation by timestamp. (all the clicks, etc. that the costumer has done in a session)

in order to measure if our baseline module are predicting correctly, one of the recommendations should have been clicked, added to chart or ordered in the y_test.

For each session in the test data, your task it to predict the aid values for each type that occur after the last timestamp ts the test session. In other words, the test data contains sessions truncated by timestamp, and you are to predict what occurs after the point of truncation.

In [100]:
df_train

session,aid,ts,type
i32,i32,i32,u8
0,1517085,1659304800,0
0,1563459,1659304904,0
0,1309446,1659367439,0
0,16246,1659367719,0
0,1781822,1659367871,0
0,1152674,1659367885,0
0,1649869,1659369893,1
0,461689,1659369898,1
0,305831,1659370027,2


In [ ]:
X = []

for session in df_train:
    x += clicks, added to charts or ordered
    if x == one of the recommendation:
        y is correct predicted
    else failier
